In [ ]:
1mport pandas as pd

from IPython.display import display

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import (
    OneHotEncoder, MinMaxScaler
)

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, auc
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from itertools import cycle

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier


import warnings
warnings.filterwarnings("ignore")

In [5]:

# Load the dataset
df = pd.read_csv(r"D:\tesis_u_sabana\data\results\contracts_colombia__analysis.csv", sep="~", encoding='utf-8')

In [6]:
df.head(9)

id_contrato                                     nombre_entidad  \
0  co1.pccntr.3402292                    municipio de san jose de cucuta   
1  co1.pccntr.2494789  sena regional quindio grupo de apoyo administr...   
2  co1.pccntr.2993001                                municipio el retiro   
3  co1.pccntr.3361440               ministerio de educacion nacional men   
4  co1.pccntr.3463351  subred integrada de servicios de salud norte e...   
5  co1.pccntr.4524505               comision nacional del servicio civil   
6  co1.pccntr.2739911                              gobernacion de narino   
7  co1.pccntr.6412677                             alcaldia de villamaria   
8  co1.pccntr.2377923  unidad administrativa especial de gestion de r...   

   nit_entidad_x estado_contrato              departamento_x     ciudad_x  \
0      890501434         cerrado          norte de santander       cucuta   
1      899999034         cerrado                     quindio      armenia   
2      890983674       terminado                   antioquia       retiro   
3      899999001         cerrado  distrito capital de bogota       bogota   
4      900971006       terminado  distrito capital de bogota  no definido   
5      900003409          cedido  distrito capital de bogota  no definido   
6      800103923       terminado                      narino        pasto   
7      890801152         cerrado                      caldas   villamaria   
8      900498879       terminado  distrito capital de bogota       bogota   

         orden                             sector                  rama  \
0  territorial                   servicio publico             ejecutivo   
1     nacional                            trabajo             ejecutivo   
2  territorial                   servicio publico             ejecutivo   
3     nacional                 educacion nacional             ejecutivo   
4  territorial          salud y proteccion social  corporacion autonoma   
5     nacional              no aplicano pertenece  corporacion autonoma   
6  territorial              no aplicano pertenece             ejecutivo   
7  territorial                   servicio publico             ejecutivo   
8     nacional  inclusion social y reconciliacion             ejecutivo   

  entidad_centralizada_x  ...    capital d_a_capital poblacion_m  va_2022_a1  \
0           centralizada  ...     cucuta    0.000000      770173  526.333254   
1        descentralizada  ...    armenia    0.000000      302613  331.158187   
2           centralizada  ...   medellin   25.025513       24466  165.286471   
3           centralizada  ...     bogota    0.000000     7732161  500.226034   
4        descentralizada  ...     bogota    0.000000     7732161  500.226034   
5           centralizada  ...     bogota    0.000000     7732161  500.226034   
6           centralizada  ...      pasto    0.000000      402073  299.310877   
7           centralizada  ...  manizales   17.802922       66914  160.281623   
8           centralizada  ...     bogota    0.000000     7732161  500.226034   

     va_2022_a2     va_2022_a3        va_2022 va_%_d_2022      %_nbi  \
0   1933.655458    9264.157243   11724.145954   53.747921  13.750122   
1    795.167698    4685.251384    5811.577269   51.169375   5.376065   
2     52.463445     627.259692     845.009607    0.441314   4.609102   
3  40817.177921  275000.089160  316317.493116  100.000000   3.471188   
4  40817.177921  275000.089160  316317.493116  100.000000   3.471188   
5  40817.177921  275000.089160  316317.493116  100.000000   3.471188   
6    858.634281    6433.293781    7591.238939   37.492337   8.244979   
7     50.759991     607.958657     819.000271    3.743355   6.861228   
8  40817.177921  275000.089160  316317.493116  100.000000   3.471188   

    tipo_de_modificacion  
0            sin_cambios  
1            sin_cambios  
2  modificacion_riesgosa  
3  modificacion_riesgosa  
4  modificacion_riesgosa  
5            sin_cambios  
6            sin

In [7]:
#preprocess the data
# Drop unnecessary columns
df.drop(columns=['id_contrato', 'documento_proveedor','nit_entidad_x','va_2022_a1', 'fecha_de_inicio_del_contrato',
                  'fecha_de_fin_del_contrato','va_2022_a2', 'va_2022_a3', 'va_2022', 'va_%_d_2022','valor_facturado',
                  'valor_pendiente_de_pago','valor_de_pago_adelantado','valor_pendiente_de_ejecucion','codigo_proveedor',
                  'id_del_proceso', 'id_del_portafolio','dominant_topic_y','valor_del_contrato_y','codigo','departamento_y', 'latitud',
                  'longitud', 'codigo_capital', 'capital','estado_contrato','valor_del_contrato_x','urlproceso_x','ciudad_y','estado_resumen'], inplace=True)

#drop index
df.drop(df.index[0], inplace=True)

#Scale the data that hasnt been scaled yet
scaler = MinMaxScaler()
df[['poblacion_m']] = scaler.fit_transform(df[['poblacion_m']])
df[['%_nbi']] = scaler.fit_transform(df[['%_nbi']])
df[['d_a_capital']] = scaler.fit_transform(df[['d_a_capital']])

In [8]:
#get X and y
y = df['tipo_de_modificacion']
X = df.drop(columns=['tipo_de_modificacion'])

#one hot encoding
# Define which columns to encode (categorical ones)
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# Create the transformer
encoder = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'  # Keep the rest of the columns (e.g., numerical)
)

# Fit and transform the data
X_encoded = encoder.fit_transform(X)

X_PCA = PCA(n_components=5).fit_transform(X_encoded)

In [9]:
def mccv_evaluation(learner, X, y, repeats=10, train_size=0.7, pipeline_name=None):
    print("Performing Monte Carlo Cross Validation for Evaluation")
    print(f"Repeats: {repeats}")
    label_order = np.unique(y)
    conf_matrix_sum = None
    f_score_sum = 0.000001

    # Prepare for ROC aggregation
    all_y_true = []
    all_y_prob = []

    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    fig.suptitle(f'MCCV Evaluation - {pipeline_name}')

    color_map = {
        "sin_cambios": "green",
        "modificacion_riesgosa": "red",
        "modificacion_general": "gold"
    }

    for i in range(repeats):
        print(f"Iteration {i + 1} of {repeats}")
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, train_size=train_size, stratify=y, random_state=i
        )
        learner.fit(X_train, y_train)
        y_pred = learner.predict(X_val)

        # Confusion matrix
        cm = confusion_matrix(y_val, y_pred, labels=label_order)
        conf_matrix_sum = cm if conf_matrix_sum is None else conf_matrix_sum + cm

        f_score_sum += f1_score(y_val, y_pred, average="weighted", labels=label_order)

        try:
            y_prob = learner.predict_proba(X_val)
            all_y_true.extend(y_val)
            all_y_prob.extend(y_prob)
        except AttributeError:
            print("Warning: predict_proba not available.")

    # Plot confusion matrix
    avg_conf_matrix = conf_matrix_sum / repeats
    avg_confusion_df = pd.DataFrame(avg_conf_matrix, index=label_order, columns=label_order)
    sns.heatmap(avg_confusion_df, annot=True, cmap="Blues", fmt=".2f", cbar=True,
                xticklabels=label_order, yticklabels=label_order,
                linewidths=0.5, linecolor="grey", ax=ax1,
                vmin=0, vmax=(len(y_val)/3)*1.1,
                cbar_kws={"shrink": .8})

    ax1.set_xlabel("Predicted Label")
    ax1.set_ylabel("True Label")
    ax1.set_title("Average Confusion Matrix")
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
    ax1.tick_params(axis='x', labelsize=10)
    ax1.tick_params(axis='y', labelsize=10)

    print(f"Average F1 Score: {f_score_sum/repeats:.4f}")

    # Plot ROC Curve using aggregated predictions
    try:
        all_y_true_bin = label_binarize(all_y_true, classes=label_order)
        all_y_prob = np.array(all_y_prob)

        for j, class_label in enumerate(label_order):
            RocCurveDisplay.from_predictions(
                all_y_true_bin[:, j],
                all_y_prob[:, j],
                ax=ax2,
                name=f"Class {class_label}",
                color=color_map.get(class_label, "gray")
            )
    except Exception as e:
        print(f"Could not plot ROC: {e}")

    ax2.plot([0, 1], [0, 1], color='k', lw=1, linestyle='--')
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.05])
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('Multi-class ROC Curve (One-vs-Rest)')
    ax2.legend(loc="lower right")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    return avg_confusion_df


In [10]:

model = KNeighborsClassifier(
    n_neighbors=5,
    n_jobs=-1
)

avg_conf_matrix = mccv_evaluation(model, X_PCA, y, repeats=10, pipeline_name="ANALYSIS KNN Classifier with PCA")

display(avg_conf_matrix)
# Save the average confusion matrix to a CSV file   
avg_conf_matrix.to_csv('average_confusion_matrix.csv', index=True, sep=',')


Performing Monte Carlo Cross Validation for Evaluation
Repeats: 10
Iteration 1 of 10
Iteration 2 of 10
Iteration 3 of 10
Iteration 4 of 10
Iteration 5 of 10
Iteration 6 of 10
Iteration 7 of 10
Iteration 8 of 10
Iteration 9 of 10
Iteration 10 of 10
Average F1 Score: 0.5946


modificacion_general  modificacion_riesgosa  \
modificacion_general                37584.0                36264.9   
modificacion_riesgosa               28565.5               169711.9   
sin_cambios                         23732.3                75540.8   

                       sin_cambios  
modificacion_general       25944.1  
modificacion_riesgosa      63420.6  
sin_cambios               167050.9

In [11]:
#train the best model with all the data
model.fit(X_PCA, y)
print("Training done")

import joblib
# Assuming 'model' is your trained model object
joblib.dump(model, 'model_1_knn.pkl')
print("Model saved!")

Training done
Model saved!
